# Zajęcia 1 (część 1)

Zakres:
* Zapoznanie sie ze srodowiskiem pracy (GKE na GCP)
* Operacje na obiektowym systemie plików (GCS)
* Interaktywna analiza rozproszonych danych przy wykorzystaniu modułu Spark Core i Spark SQL
* Wprowadzenie modułu pandas
* Wizualizacje przy wykorzystaniu matplotlib oraz seaborn

Języki:
* bash, Python, SQL 

Krótka lista przydatnych poleceń bash: https://www.reddit.com/r/linux/comments/9rns12/some_linux_commands_cheatsheet/

### Jupyter Notebook

* Interaktywny notatnik dostepny poprzez przeglądarkę.
* Służy do wpisywania poleceń w wybranych jezykach programowania oraz opis w tzw jezyku markdown.
* Polecenia/opis wpisujemy do komórek. Komórki są wykonywane przez tzw. kernel (np Python w określonej wersji)
* Działa w tryb edycji komórek + tryb wykonywania poleceń. (przejście poprzez ESC i ENTER)
* Podstawowe skróty klawiaturowe: 
  * Esc/Enter
  * strzalki
  * ctrl-enter/shift-enter
  * a/b/d
  * y/m 
* Moze wykonywać polecenia powłoki (bash) !  %%bash
* Notatnik zapisywany jest w formacie ipynb (IPythonNoteBook)
* Kolejnosc uruchomienia komórek moze byc rozna.
* Mozliwosc zatrzymania kernela i uruchomienia od nowa. 

Polecamy film: https://www.youtube.com/watch?v=HW29067qVWk

Przykładowe komórki Pythonowe i Bashowe:

In [2]:
import os
user = os.environ.get('JUPYTERHUB_USER')
msg = f"Hello {user}!"
print (msg)

Hello agaszmurlo!


In [3]:
! pwd # pokaż aktualną scieżkę

/home/jovyan/ds-notebooks/session_1


In [14]:
%%bash
head cloud.ipynb
echo "--------------------"
tail cloud.ipynb

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Środowisko Google Kubernetes Engine \n",
    "\n",
    "Logowanie kontem github. \n",
    "\n",
--------------------
   "version": "3.8.5"
  },
  "notebook_test": {
   "keytab_path": "/data/work/home/ds-lab-testuser1/ds-lab-testuser1.keytab",
   "user": "ds-lab-testuser1"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


## Kopiowanie danych do swojego katalogu domowego

W katalogu `/mnt/data/datascience` znajduje sie plik który bedzie nam dzisiaj słuzyl do pracy. Należy go skopiować do swojego katalogu domowego. 
Dane pochodzą z https://gdac.broadinstitute.org/ i zawierają dane z badaniami nad mutacjami typu CNV w obrebie genu BRAC2.


In [1]:
%%bash
mkdir -p ~/data     # stworz katalog data (i posrednie)
cp /mnt/data/datascience/brca.txt ~/data  # skopiuj plik z lokalizacji do bieżącej lokalizacji
ls -lah   ~/data       # wylistuj zawartość bieżącego katalogu


total 376K
drwxrwsr-x  2 jovyan users 4.0K Apr 21 13:26 .
drwxrwsr-x 25 root   users 4.0K Apr 21 13:26 ..
-rw-rw-r--  1 jovyan users 367K Apr 21 13:39 brca.txt


In [ ]:
%%bash
head ~/data/brca.txt   # pokaż początek pliku
echo                   # pusta linia
tail ~/data/brca.txt   # pokaż koniec pliku 
echo
wc -l ~/data/brca.txt  # policz linie pliku

# Środowisko Google Kubernetes Engine 

Logowanie kontem github. 

🙏  Podziękowania dla Operatora Chmury Krajowej (https://chmurakrajowa.pl/) i Google Poland za udostępnienie infrastruktury i pokrycie kosztów wykorzystania zasobów podczas zajęć. 🙏 


## Kubernetes

Kubernetes (k8s) to otwarta platforma do koordynacji wysoko dostępnego klastra. 
* Umożliwia  deklaratywną konfigurację, automatyzację wdrażania, skalowanie i autoskalowanie rozwiązań.
* Pozwala uruchamiać aplikacje/narzędzie bez przypisywania ich do konkretnej maszyny. Aplikacje muszą być niezależne od konkretnego serwera: muszą być skonteneryzowane.
* Możliwe do uruchomienia na prywatnym centrum danych, infrastrukturze hybrydowej lub chmurze publicznej.


**Klaster Kubernetes**  
* **Węzeł sterujący (Master node)** koordynuje działanie klastra np. zlecanie uruchomienia aplikacji, utrzymywanie pożądanego stanu aplikacji, skalowanie aplikacji i instalowanie nowych wersji
* **Węzeł roboczy (Worker node)** Na węzłach uruchamiane są aplikacje. Na każdym węźle działa agent zarządzający tym węzłem i komunikujący się z masterem Kubernetes (API). Węzeł zawiera także narzędzia do obsługi kontenerów

**Pod** to grupa jednego/wielu kontenerów wraz ze wspólnymi zasobami (np. dysk). Pod tworzy "wirtualney serwer" i może zawierać różne kontenery aplikacji współdzielące zasoby i kontekst wykonawczy na tym samym węźle.
* Pod jest uruchamiany na węźle roboczym. Węzeł jest maszyną roboczą, fizyczną lub wirtualną, w zależności od klastra. 
* Węzeł może zawierać wiele podów. 
* Kubernetes master automatycznie zleca uruchomienie podów na różnych węzłach w ramach klastra. Automatyczne zlecanie uruchomienia bierze pod uwagę zasoby dostępne na każdym z węzłów.

![](../img/gke.png) 


## Wykorzystanie Google Cloud Storage

#### Projekt
* Wszystki dane przynależą do konkretnego projektu.
* Do projektu mogą mieć dostęp użytkownicy. 
* Projekt ma zdefiniowane metody uwierzytelniające, rozliczenia, monitorowanie etc. 
#### Kubełek (bucket)
* Kubełek (buckket) to kontener na pliki/obiekty. 
* Nazwa Bucketu musi być unikalna w skali całej usługi u wszystkich użytkowników (!) 
* Kubełków nie można zagnieżdzać
* W kubełkach możemy tworzyć foldery i tam logicznie grupować pliki.
* Kubełek wraz z zawartością może zostać udostępniony publicznie.
* Kubełkowi nie można zmienić nazwy lub metadanych. Trzeba go usunąć i stworzyć ponownie.
#### Obiekt 
* obiekty przechowywane w kubełkach 
* obiekty mają zawartość oraz metadane
* obiekty są niemodyfikowalne 

Do operacji na Google Storage można wykorzystać narzędzie `gsutil`: 

##### Operacje na kubełkach
* listowanie kubełków (buckets) - `ls`
* tworzenie nowego kubełka - `mb`
* usuwania kubełka - `rm`
* listowanie zawartości kubełków - `ls` 
* udostępnianie - `iam`

##### Operacje na obiektach
* dodawania pliku do kubełka - `cp`
* kopiowanie - `cp`
* usuniecie z kubełka - `cp`
* pobranie informacji o obiekcie - `stat`

#### Operacje na kubełkach

In [22]:
! gsutil ls

gs://artifacts.studiapodyplomowe.appspot.com/
gs://bdg-lab-agaszmurlo/
gs://bdg-lab-bwentura/
gs://bdg-lab-gcr/
gs://bdg-lab-karolinaow/
gs://bdg-lab-mandrush/
gs://bdg-lab-matdaw/
gs://bdg-lab-michalrudko/
gs://bdg-lab-mszelest/
gs://bdg-lab-mwiewior/
gs://bdg-lab-tgambin/
gs://bdg-sequila-dev/
gs://bucket-agaszmurlo/
gs://bucket-aniaszczepinska/
gs://bucket-dd004/
gs://bucket-ds-lab-testuser1/
gs://bucket-edugen/
gs://bucket-karolina95/
gs://bucket-kipk29/
gs://bucket-lutram/
gs://bucket-matdaw/
gs://bucket-mp0028/
gs://bucket-nataliawu/
gs://bucket-t-h-i-n-k/
gs://bucket-tgambin/
gs://ds_labs_agaszmurlo/
gs://edugen-common-data/
gs://edugen-data/
gs://edugen-lab-agaszmurlo/
gs://edugen-lab-hallay/
gs://edugen-lab-matdaw/
gs://edugen-lab-tgambin/
gs://eu.artifacts.studiapodyplomowe.appspot.com/
gs://tbd-infra/
gs://tbd-project/
gs://test_agaszmurlo/
gs://tgambin-hidden/


In [ ]:
! echo $USER  # wyswietlenie nazwy uzytkownika

In [ ]:
! gsutil mb gs://bucket-$USER  # stworzenie bucketu

In [ ]:
! gsutil ls -L -b gs://bucket-$JUPYTERHUB_USER # listowanie zawartości 

In [ ]:
! gsutil du -s  gs://bucket-$USER # ile zajmuje przestrzeni?

#### Operacje na zawartości kubełków

In [ ]:
! gsutil ls -r gs://bucket-$USER/ # listowanie zawartosci kubełka

In [ ]:
! gsutil cp ~/work/git/ds-notebooks/README.md gs://bucket-$USER  # upload obiektu do kubełka

In [ ]:
! gsutil ls -r gs://bucket-$USER # listowanie zawartości

In [ ]:
! gsutil iam get gs://bucket-$USER

In [ ]:
! gsutil stat gs://bucket-$USER/README.md # metadane obiekty w kubełku

In [ ]:
! gsutil iam ch allUsers:objectViewer gs://bucket-$USER # dodanie uprawnien do odczytu

Po wykonaniu tego polecenia nasz kubełek staje się publiczny i możemy się do niego 
http://storage.googleapis.com/bucket-NAZWA_UZYTKOWNIKA. Zawartość pliku można odczytać poprzez http://storage.googleapis.com/bucket-NAZWA_UZYTKOWNIKA/NAZWA_OBIEKTU

In [ ]:
! gsutil iam ch -d allUsers:objectViewer gs://bucket-$USER # usuniecie uprawnien do odczytu

Teraz można ponownie zweryfikowac możliwość publicznego odczytu danych z kubełka.

#### Przeniesienie do docelowego kubełka

In [ ]:
%%bash
gsutil cp -r gs://bucket-$USER gs://DOCELOWY_BUCKET
gsutil rm -r gs://bucket-$USER

## <span style='background:yellow'> ZADANIE 1 </span>
a) Utwórz plik zawierający aktualną date i godzinę. 
b) Udostępnij plik publicznie na google storage. 

## <span style='background:yellow'> ZADANIE 2 </span>
Skopiuj plik z lokalizacji `gs://bucket-$USER` do lokalizacji `gs://bdg-lab-$USER`.

### Pobranie pliku z powrotem na lokalny dysk

In [23]:
! gsutil cp gs://bucket-$USER/brca.txt .  # gdzie sie zapisal ten plik?

Copying gs://bucket-agaszmurlo/brca.txt...
/ [1 files][366.8 KiB/366.8 KiB]                                                
Operation completed over 1 objects/366.8 KiB.                                    


## Apache Spark

* Apache Spark platforma ogólnego zastosowania, opensource, do przetwarzania duzych zbiorow danych.
* Posiada  API dla języków programowania: Scala, Python i R. 
* Przetwarzanie w Spark przetwarzanie jest wykonywane w większości  wprost w pamięci operacyjnej.
* Przeznaczenie: do uruchamiania  aplikacji i skryptów z wykorzystaniem uczenia maszynowego lub interaktywnych kwerend.
* Spark ten wspiera SQL (typ DataFrames), przetwarzanie strumieniowe oraz przetwarzanie grafów.
* Integracja z lokalną pamięci masową, rozproszonymi lub obiektowymi systemu plików.
* Spark można uruchamić na pojedynczej maszynie na środowisku klastrowym, lub w chmurze. 

Zakres na dzisiejsze laboratorium:
* stworzenie sesji Spark
* zaczytanie danych z pliku tekstowego
* kwerendy na danych

Bedziemy korzystac z pliku ktory zapisalismy na HDFS w pierwszej czesci cwiczenia.

In [ ]:
! hdfs dfs -ls /user/${USER}/external/data # Sprawdzmy czy plik znajduje się w katalogu

### Przygotowanie sesji Sparkowej


In [2]:
import os
user_name = os.environ.get('USER')
print(user_name)

agaszmurlo


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.config("spark.executor.instances", "1") \
.config("spark.executor.memory", "1g") \
.getOrCreate()

### Ręczne utworzenie zbioru danych

In [7]:
df_users = spark.createDataFrame(
    [(123, 'Jan Kowalski', 'jan@kowalski.pl'), 
     (124, 'Anna Nowak', 'anna@nowak.pl'), 
     (125, 'Janusz Kowalski', 'janusz@kowalski.pl'), 
     (126, 'Anita Nowak', 'anita@nowak.pl')])

In [9]:
df_users.show()

+---+---------------+------------------+
| _1|             _2|                _3|
+---+---------------+------------------+
|123|   Jan Kowalski|   jan@kowalski.pl|
|124|     Anna Nowak|     anna@nowak.pl|
|125|Janusz Kowalski|janusz@kowalski.pl|
|126|    Anita Nowak|    anita@nowak.pl|
+---+---------------+------------------+



In [24]:
df_users.printSchema()

root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)



In [21]:
df_users = spark.createDataFrame(
    [(123, 'Jan Kowalski', 'jan@kowalski.pl'), 
     (124, 'Anna Nowak', 'anna@nowak.pl'), 
     (125, 'Janusz Kowalski', 'janusz@kowalski.pl'), 
     (126, 'Anita Nowak', 'anita@nowak.pl')],
    schema='id int not null, name string, email string')

In [22]:
df_users.show()

+---+---------------+------------------+
| id|           name|             email|
+---+---------------+------------------+
|123|   Jan Kowalski|   jan@kowalski.pl|
|124|     Anna Nowak|     anna@nowak.pl|
|125|Janusz Kowalski|janusz@kowalski.pl|
|126|    Anita Nowak|    anita@nowak.pl|
+---+---------------+------------------+



In [23]:
df_users.printSchema()

root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)




### Wczytanie danych
Zazwyczaj dane mamy przechowane na zewnątrz naszego notatnika (np. w pliku) i należy je odczytać.

In [7]:
input_path = f'gs://bdg-lab-{user_name}/brca.txt'

In [8]:
df = spark.read.load(input_path, format="csv", sep="\t", inferSchema="true", header="true")

### Charakterystyka danych

In [ ]:
type (df)                 # jaki jest typ danych

In [ ]:
df.explain()              # fizyczny plan wykonania

In [ ]:
df.explain(True)          # logiczny i fizyczny plan wykonania

In [ ]:
df.rdd.getNumPartitions() # liczba partycji (bloków danych)

In [ ]:
df.printSchema()          # schemat danych

In [ ]:
df.count()                # wymiary (liczba wierszy)

In [ ]:
len(df.columns)           # wymiary (liczba kolumn)

In [ ]:
df.describe().show()      # pokaż podsumowanie danych w tabeli

In [ ]:
df.describe("Chromosome").show() # pokaż podsumowanie danych w kolumnie 

### Odczyt danych

In [ ]:
df.show()                    # pokaż 20 wierszy danych 

In [ ]:
df.show(10, truncate=False) # pokaż 10 wierszy, nie skracaj danych

In [ ]:
df.select("Sample").show()  # pokaż tylko kolumne sample

In [ ]:
df.select("Chromosome", "Start", "End").show() # pokaż kolumny chromosome, start i end

In [ ]:
df_chrom = df.select(df.Chromosome).distinct() # stworz nowy DF zawierajacy tylko unikalne wartosci chromosomow

In [ ]:
df_chrom.count()                     # policz wiersze w nowym DF

In [ ]:
df.filter("Chromosome > 21").show() # pokaz dane spelniajace warunek

In [ ]:
df.filter("Chromosome > 21").explain()  # pokaz plan wykonania

In [ ]:
df.filter("Chromosome > 21 and Segment_Mean > 0").show() # pokaz dane spelniajace warunki

### Grupowanie i funkcje agregujące

In [10]:
from pyspark.sql.functions import *

In [ ]:
df.groupBy("Chromosome").count().show()   # dokonaj grupowania po chromosomie i policz rekordy w grupie

In [ ]:
df.groupBy("Chromosome").avg("Segment_Mean").show() # dokonaj grupowania po chromosomie i policz srednia wartosc segment_mean w grupie

In [ ]:
# pogrupuj dane wzgledem probki i chromosomu, policz rekordy w grupie
df.groupBy("Sample","Chromosome").count().orderBy(asc("Sample")).show()

### Kolumny wyliczane

In [ ]:
df.withColumn ("Length", col("End") - col("Start")).show() # dodaj kolumne dlugosc jako end - start

In [ ]:
df.withColumn ("Material", lit("DNA")).show()     # dodaj kolumne o stalej wartosci 'DNA'

In [ ]:
df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome"))).show() # dodaj kolumne z konkatencja

In [ ]:
df.drop("Chromosome").show()  # usun kolumne Chromosome. 
# Czy DF została pozbawiona kolumny na trwale?

### Zapis wyników

In [14]:
df2 = df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome")))  # dodaj kolumne z konkatenacja i zapisz do nowego DF

In [15]:
output_path = f'gs://bdg-lab-{user_name}/brca2.txt'
df2.write.format("csv").save(output_path)

In [16]:
! gsutil ls gs://bdg-lab-$USER

gs://bdg-lab-agaszmurlo/brca.txt
gs://bdg-lab-agaszmurlo/brca2.txt/
gs://bdg-lab-agaszmurlo/mlflow/
gs://bdg-lab-agaszmurlo/survey/


In [20]:
!gsutil cat gs://bdg-lab-$USER/brca.txt | head

Sample	Chromosome	Start	End	Num_Probes	Segment_Mean
TCGA-A2-A0EU-10A-01D-A060-02	1	10209	2583075	NA	-0.210121164888764
TCGA-A2-A0EU-10A-01D-A060-02	1	2583076	249240606	NA	-0.101789555120838
TCGA-A2-A0EU-10A-01D-A060-02	2	10002	243189359	NA	-0.337963741969504
TCGA-A2-A0EU-10A-01D-A060-02	3	60175	162511435	NA	-0.0850647472883194
TCGA-A2-A0EU-10A-01D-A060-02	3	162511436	162626067	NA	0.42309157125509
TCGA-A2-A0EU-10A-01D-A060-02	3	162626068	197962416	NA	-0.0962225847981023
TCGA-A2-A0EU-10A-01D-A060-02	4	69223	191014397	NA	-0.0713350286479217
TCGA-A2-A0EU-10A-01D-A060-02	5	11770	180905246	NA	0.176064461471459
TCGA-A2-A0EU-10A-01D-A060-02	6	63815	171050932	NA	-0.0895478602860139


## <span style='background:yellow'> ZADANIE 3</span>

a) Ile jest unikalnych danych próbek w tym zbiorze danych?

b) Pokaż nazwę próbki, numer chromosomu, początek i koniec segmentu dla segmentów występujących na chromosomie 21,22,23 i mających startową pozycję większą niż 10000000. 

c) Ile występuje wierszy dla każdej z próbki? Pokaż wynik z pełną nazwą próbki. Zapisz wynik do pliku na GCS